In [359]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.experimental import enable_hist_gradient_boosting

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import GammaRegressor
from sklearn.linear_model import LogisticRegression

from joblib import dump, load
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler
import pandas as pd
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_columns', None)

In [360]:
df=pd.read_parquet('../data/processed/df.parquet')
dfd=pd.read_parquet('../data/processed/dfd.parquet')


In [361]:
dfd.columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'df_original', 'id',
       'cut_int', 'color_int', 'clarity_int', 'cut_color_clarity_score',
       'raw_density', 'marketing_carat_int', 'carat_sqr', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'marketing_carat_bad',
       'marketing_carat_good', 'marketing_carat_regular'],
      dtype='object')

In [362]:
#option 1

# dfd=dfd.drop(columns=['marketing_carat_int','cut_int','color_int', 'clarity_int'])

# #option 2
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score'])

# #option 3 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z'])

# #option 3 bis espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density'])

# #option 3 bis2 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'x', 'y', 'z'])

# #option 4
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z','raw_density'])

# #option 5 SIN NADA
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density','marketing_carat_bad',
#        'marketing_carat_good', 'marketing_carat_regular'])

#option 5 SIN NADA
dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
                     'marketing_carat_bad',
       'marketing_carat_good', 'marketing_carat_regular','carat_sqr','cut_color_clarity_score'])

In [363]:
#con carat y 5
# RandomForestRegressor       536.7894862874348
# ExtraTreesRegressor         524.3073747284154
# GradientBoostingRegressor   522.2696670624822
# CatBoostRegressor           515.8518906580055
# LGBMRegressor               520.4246279374379
# XGBRegressor                517.9044507009544
# StackingRegressor           504.8044508186878

#sin carat y 5
# RandomForestRegressor       536.5326496454497
# ExtraTreesRegressor         522.8754833840452
# GradientBoostingRegressor   521.8840694917378
# CatBoostRegressor           514.4646192634776
# LGBMRegressor               520.4418909188064
# XGBRegressor                517.6058680138955
# StackingRegressor           504.43703022069695


df=df.drop(columns=['cut','color', 'clarity','marketing_carat','marketing_carat_int','cut_color_clarity_score','carat_sqr'])


#RandomForestRegressor        536.8794322366753
# ExtraTreesRegressor         522.5880641009147
# GradientBoostingRegressor   521.8113311805382
# CatBoostRegressor           515.7005012415111
# LGBMRegressor               520.4246279374379
# XGBRegressor
# StackingRegressor           505.6372856024203

# df=df.drop(columns=['cut','color', 'clarity','marketing_carat','marketing_carat_int','cut_color_clarity_score','carat_sqr'])



In [364]:
df.head()


,carat,depth,table,price,x,y,z,df_original,id,cut_int,color_int,clarity_int,raw_density
0,1.21,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,4,1,4,3.683471
1,0.32,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,3,3,4,3.664419
2,0.71,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,1,4,5,3.755392
3,0.41,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,2,7,3,3.712154
4,1.02,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,5,4,3,3.633558


In [365]:
dfd.head()

,carat,depth,table,price,x,y,z,df_original,id,raw_density,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,3.683471,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,0.32,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,3.664419,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,0.71,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,3.755392,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,0.41,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,3.712154,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1.02,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,3.633558,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


## Split

In [366]:


df_final=df

data=df_final.loc[df['df_original']==1]

############## FILAS

# def remove_outlier(df, col):
#     q1 = df[col].quantile(0.25)
#     q3 = df[col].quantile(0.75)
#     iqr = q3-q1 
#     fence_low  = q1-1.5*iqr
#     fence_high = q3+1.5*iqr
#     df_clean = df.loc[(df[col] > fence_low) & (df[col] < fence_high)]
#     return df_clean

# data=data.drop_duplicates(subset=data.columns, keep='first')

data['depth_calc'] = data['z'] / ((data['x'] + data['y'])/2)*100
data['rest']=data['depth_calc'] - data['depth'] 
filter_rest=(data['rest']<-1) | (data['rest']>1)
data=data.loc[~filter_rest]
data=data.drop(columns=['depth_calc','rest'],axis=1)

filter_xyz=(data['x']<0.2) | (data['y']<0.2) | (data['z']<0.2)
data=data.loc[~filter_xyz ]



df_predict=df_final.loc[df['df_original']==0]

X=data.drop(columns=['price','df_original','id'])
Y=data['price']

X_predict=df_predict.drop(columns=['price','df_original','id'])



# rs= StandardScaler()

# rs = RobustScaler() 

# X = pd.DataFrame(rs.fit_transform(X), columns= X.columns)
# X_predict = pd.DataFrame(rs.fit_transform(X_predict), columns= X_predict.columns)


df_predict.to_csv('../data/processed/df_predict.csv',index=False)
X.to_csv('../data/processed/X.csv',index=False)
Y.to_csv('../data/processed/Y.csv',index=False)
X_predict.to_csv('../data/processed/X_predict.csv',index=False)


print(5)
print(data.shape)
print(df_predict.shape)
print(X.shape)
print(X_predict.shape)
display(X)
display(X_predict)


/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


5
(40380, 13)
(13485, 13)
(40380, 10)
(13485, 10)


,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,raw_density
0,1.21,62.4,58.0,6.83,6.79,4.25,4,1,4,3.683471
1,0.32,63.0,57.0,4.35,4.38,2.75,3,3,4,3.664419
2,0.71,65.5,55.0,5.62,5.53,3.65,1,4,5,3.755392
3,0.41,63.8,56.0,4.68,4.72,3.00,2,7,3,3.712154
4,1.02,60.5,59.0,6.55,6.51,3.95,5,4,3,3.633558
...,...,...,...,...,...,...,...,...,...,...
40450,1.34,62.7,57.0,7.10,7.04,4.43,5,4,5,3.630959
40451,2.02,57.1,60.0,8.31,8.25,4.73,2,5,2,3.737546
40452,1.01,62.7,56.0,6.37,6.42,4.01,5,3,3,3.695335
40453,0.33,61.9,54.3,4.45,4.47,2.76,5,1,5,3.606522


,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,raw_density
0,0.79,62.7,60.0,5.82,5.89,3.67,3,5,3,3.767680
1,1.20,61.0,57.0,6.81,6.89,4.18,5,1,5,3.671046
2,1.57,62.2,61.0,7.38,7.32,4.57,4,3,3,3.815639
3,0.90,63.8,54.0,6.09,6.13,3.90,3,5,3,3.708953
4,0.50,62.9,58.0,5.05,5.09,3.19,3,5,5,3.658655
...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,5,6,3,3.641219
13481,0.71,62.2,55.0,5.71,5.73,3.56,5,2,4,3.657370
13482,0.70,61.6,55.0,5.75,5.71,3.53,5,5,5,3.623853
13483,0.70,58.8,57.0,5.85,5.89,3.45,3,5,2,3.533125


## Model Generator

In [367]:
# arboles  de decision max_depth probar min sample split
# RandomForestRegressor(max_depth=40, min_samples_split=4, n_estimators=500, random_state=32)
# ('RandomForestRegressor', RandomForestRegressor(max_depth=32, min_samples_split=4, n_estimators=250, random_state=32)),
# max_depth=-1


estimators=[('XGBRegressor' , XGBRegressor(base_score=0.8 ,booster = 'gbtree' ,learning_rate=0.05,max_depth= 6 ,n_estimators=500 ,n_jobs = -1)),
    ('LGBM' , LGBMRegressor(learning_rate =  0.05, n_estimators =  435, max_depth=12,n_jobs = -1)),
             ('ExtraTreesRegressor',ExtraTreesRegressor(max_depth=16, min_samples_split=6, n_estimators=400,n_jobs = -1)),
           ('GradientBoostingRegressor',GradientBoostingRegressor(max_depth=6, min_samples_leaf=6, min_samples_split=16, n_estimators=200,n_jobs = -1)),
            ('CatBoostRegressor',CatBoostRegressor(depth = 8, iterations = 1200,silent=True,n_jobs = -1)),
            
        ]



machine = [
           ExtraTreesRegressor(max_depth=16, min_samples_split=6, n_estimators=400,n_jobs = -1),
           GradientBoostingRegressor(max_depth=6, min_samples_leaf=6, min_samples_split=16, n_estimators=200,n_jobs = -1),
           CatBoostRegressor(depth = 8, iterations = 1200,silent=True,n_jobs = -1),
           LGBMRegressor( learning_rate =  0.05, n_estimators =  435, max_depth=12,n_jobs = -1),
           XGBRegressor(base_score=0.8 ,booster = 'gbtree' ,learning_rate=0.05,max_depth= 6 ,n_estimators=500,n_jobs = -1),
          StackingRegressor(estimators=estimators,final_estimator= LinearRegression(n_jobs = -1),
                               cv= 5 , verbose = 10 , passthrough = True),
          ]  


machine = [  StackingRegressor(estimators=estimators,final_estimator= LinearRegression(n_jobs = -1) ,cv= 5 , 
                               verbose = 2 , passthrough = True)]  

# LinearRegression(n_jobs = -1)
# ARDRegression(n_iter=1000)

# machine = [ GradientBoostingRegressor(max_depth=6, min_samples_leaf=6, min_samples_split=16, n_estimators=200,random_state=32) ]


# lgbm  boosting_type='rf', 
#                        bagging_freq=1, 
#                        bagging_fraction = 0.9





for x in machine:
    print(f'Model in action: {x}\n')
    
#     X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.85, random_state=32)
    
######################           ENTRENAR AL FINAL CON TODO


        
        
#     model = x  
#     model.fit(X_train, Y_train)
    
    
#     predictions_train = model.predict(X_train)#.clip(326,18823)
#     predictions_test = model.predict(X_test)#.clip(326,18823)
 

    
#     print(f' R2 Train: {r2_score(y_true=Y_train, y_pred=predictions_train)}')
#     print(f' R2 Test: {r2_score(y_true=Y_test, y_pred=predictions_test)}')
#     print(f' RMSE = {(mean_squared_error(Y_test, predictions_test,squared=False))}\n\n')
    
    
    model = x  

    
    
#     model.fit(X,Y)


#     scores = cross_val_score(model, X, Y, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
#     print(f' RMSE_CROSS = {np.mean(-scores)}\n\n')



        # SinROBUSTyConRAWDENSITYyConCARATSQR2
# ExtraTreesRegressor         523.534
# GradientBoostingRegressor   517.786  
# CatBoostRegressor           516.027 
# LGBMRegressor               521.557  
# XGBRegressor                518.264 
# StackingRegressor           505.110 

        # SinROBUSTySinRAWDENSITYySinCARATSQR2
# ExtraTreesRegressor         523.1499
# GradientBoostingRegressor   520.603 
# CatBoostRegressor           514.409 
# LGBMRegressor               524.278 
# XGBRegressor                520.474 
# StackingRegressor           506.345

        # SinROBUSTySinRAWDENSITYyConCARATSQR2
# ExtraTreesRegressor         524.460
# GradientBoostingRegressor   520.599 
# CatBoostRegressor           515.747 
# LGBMRegressor               524.278 
# XGBRegressor                520.474 
# StackingRegressor           506.791

        # SinROBUSTyConRAWDENSITYySinCARATSQR2
# ExtraTreesRegressor         522.318
# GradientBoostingRegressor   517.817
# CatBoostRegressor           515.862
# LGBMRegressor               521.558
# XGBRegressor                518.264
# StackingRegressor           505.278



Model in action: StackingRegressor(cv=5,
                  estimators=[('XGBRegressor',
                               XGBRegressor(base_score=0.8, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.05,
                                            max_delta_step=None, max_depth=6,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n...
                               ExtraTreesRegressor(max_depth=16,
                                                   min_sam

KeyboardInterrupt: 

In [341]:
5



5

## RandomizedSearchCV

In [355]:
# CatBoostRegressor(depth = 8, iterations = 1200,silent=True)

model2= GradientBoostingRegressor()


#  'n_estimators': [16, 32, 64, 128, 256, 500],
#     'min_samples_split': [2, 8, 10, 12],
#     'max_depth' : [2, 4, 8, 10, 16, 32, 50, 75]
# min_samples_split
# min_samples_leaf=1
#leaves

# iterations=None,
#                         learning_rate=None,
#                         depth=None

param_grid = {
'max_depth' :[4,6,8], 
    'min_samples_split' :[14,16,18], 
    'n_estimators':[120,200,300],
    'min_samples_leaf' : [4,6,8],
}

# Randomized

grid_search = RandomizedSearchCV(model2, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                n_iter=32)
# n_iter=32
grid_search.fit(X, Y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 16.0min finished


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_iter=32,
                   n_jobs=-1,
                   param_distributions={'max_depth': [4, 6, 8],
                                        'min_samples_leaf': [4, 6, 8],
                                        'min_samples_split': [14, 16, 18],
                                        'n_estimators': [120, 200, 300]},
                   scoring='neg_root_mean_squared_error', verbose=10)

In [356]:
grid_search.best_params_

{'n_estimators': 120,
 'min_samples_split': 18,
 'min_samples_leaf': 6,
 'max_depth': 8}

In [357]:
grid_search.best_estimator_
# (learning_rate=0.05, max_depth=12, n_estimators=430,random_state=32)

GradientBoostingRegressor(max_depth=8, min_samples_leaf=6, min_samples_split=18,
                          n_estimators=120)

In [358]:
grid_search.best_score_
# -520.5383378789018

-517.8223457640372

## PROBAR RESULTADO

In [319]:
# dfX_test=pd.DataFrame(X_test)
# dfY_test=pd.DataFrame(Y_test)
# dfpredictions_test=pd.DataFrame(predictions_test)
# dftest=pd.concat([dfX_test.reset_index(drop=True),dfY_test.reset_index(drop=True),dfpredictions_test.reset_index(drop=True)], axis=1) 
# dftest.rename(columns={0: 'price_predict'},inplace=True)
# dftest['Diff']=dftest['price']-dftest['price_predict']

# print(dftest['Diff'].describe())
# print(dftest.describe())
# dftest.loc[dftest['Diff']>2000].describe()

count      1.000000
mean     859.061216
std             NaN
min      859.061216
25%      859.061216
50%      859.061216
75%      859.061216
max      859.061216
Name: Diff, dtype: float64


,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,price,price_predict,Diff
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
# dump(model, '/home/carpiero/ir/Kaggle/.joblib') 

predictions_competition = model.predict(X_predict)
submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition})
submission['id'] = submission['id'].astype('int64')

# submission.loc[94,'price'] =20631

submission.to_csv('/home/carpiero/IRONHACK/kaggle/submission.csv', index=False)

PROBA=pd.concat([X_predict,submission], axis=1)



submission


,id,price
0,0,2956.558372
1,1,5585.794483
2,2,9485.855474
3,3,4007.838958
4,4,1637.088890
...,...,...
13480,13480,1696.806954
13481,13481,2396.171682
13482,13482,3112.518978
13483,13483,2084.779690


In [343]:

# SinROBUSTyConRAWDENSITYyConCARATSQR2
# count    13485.000000
# mean      3947.999179
# std       3958.699006
# min        163.363126
# 25%        934.982225
# 50%       2464.969051
# 75%       5278.298692
# max      18560.749100
# Name: price, dtype: float64

#SinROBUSTySinRAWDENSITYySinCARATSQR2
# count    13485.000000
# mean      3947.396160
# std       3956.301710
# min        323.516320
# 25%        936.455943
# 50%       2446.378377
# 75%       5292.367311
# max      18360.274125
# Name: price, dtype: float64

# SinROBUSTySinRAWDENSITYyConCARATSQR2
# count    13485.000000
# mean      3948.926775
# std       3958.269583
# min        330.948345
# 25%        938.487300
# 50%       2455.687555
# 75%       5285.923421
# max      18534.746496
# Name: price, dtype: float64

# SinROBUSTyConRAWDENSITYySinCARATSQR2

# count    13485.000000
# mean      3947.837618
# std       3957.962843
# min        338.634208
# 25%        935.633835
# 50%       2452.391613
# 75%       5289.271769
# max      18374.874823
# Name: price, dtype: float64


PROBA['price'].describe()

count    13485.000000
mean      3947.837618
std       3957.962843
min        338.634208
25%        935.633835
50%       2452.391613
75%       5289.271769
max      18374.874823
Name: price, dtype: float64

In [344]:
PROBA.loc[(PROBA['price']>18000) | (PROBA['price']<0)  | (PROBA['carat']>3.5)  ]

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,raw_density,id,price
94,5.01,65.5,59.0,10.74,10.54,6.98,1,1,1,3.804422,94,17051.375785
9468,2.03,59.6,60.0,8.27,8.21,4.91,4,4,4,3.653567,9468,18062.160765
9724,2.29,61.4,60.0,8.45,8.49,5.20,4,1,8,3.683146,9724,18374.874823
9827,1.70,60.6,58.0,7.71,7.68,4.66,4,5,5,3.696568,9827,18036.484569
9838,3.65,67.1,53.0,9.53,9.48,6.38,1,3,1,3.799463,9838,15013.289689
11795,1.71,61.9,61.0,7.61,7.67,4.73,3,5,5,3.716261,11795,18035.902308
12057,2.00,63.7,60.0,7.88,7.81,5.00,2,7,4,3.899725,12057,18049.370077
12105,1.70,61.0,56.0,7.62,7.67,4.66,5,4,7,3.745104,12105,18116.267553
12305,3.67,62.4,56.0,9.86,9.81,6.13,4,2,1,3.713735,12305,14173.214128
12782,2.02,61.7,53.0,8.14,8.04,4.99,5,4,4,3.711262,12782,18065.789271


In [345]:
PROBA.loc[(PROBA['cut_int']==1) & (PROBA['clarity_int']==1)  & (PROBA['color_int']==1)  ]

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,raw_density,id,price
94,5.01,65.5,59.0,10.74,10.54,6.98,1,1,1,3.804422,94,17051.375785
6418,2.72,68.2,56.0,8.46,8.43,5.76,1,1,1,3.972828,6418,9919.048417
6608,0.70,64.7,59.0,5.59,5.50,3.59,1,1,1,3.805225,6608,1018.108867
6668,0.96,59.5,60.0,6.28,6.23,3.72,1,1,1,3.957600,6668,1953.133886


In [299]:
PROBA.loc[(PROBA['price']>18000) | (PROBA['price']<0)].describe()

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,id,price
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,1.990909,61.254545,59.090909,8.051818,8.087273,4.941818,3.363636,4.545455,4.636364,8264.636364,18116.607467
std,0.243494,1.295657,1.513575,0.343127,0.350345,0.210752,0.924416,1.967925,1.804036,3852.818067,124.507557
min,1.700000,58.700000,56.000000,7.610000,7.670000,4.660000,2.000000,1.000000,2.000000,1025.000000,18001.059742
25%,1.735000,60.700000,58.500000,7.760000,7.745000,4.765000,3.000000,3.500000,3.500000,5748.500000,18022.705946
50%,2.010000,61.100000,60.000000,8.030000,8.110000,5.000000,3.000000,5.000000,4.000000,9724.000000,18044.023821
75%,2.215000,61.750000,60.000000,8.380000,8.430000,5.110000,4.000000,6.000000,5.500000,11281.500000,18219.980960
max,2.290000,63.700000,61.000000,8.470000,8.500000,5.200000,5.000000,7.000000,8.000000,12105.000000,18360.274125
